# Use nn-Meter for Bench Dataset
In this notebook, we showed nn-Meter examples of latency prediction for bench dataset.

In [1]:
import os
import nn_meter
from nn_meter.dataset import bench_dataset

datasets = bench_dataset()
for data in datasets:
    print(os.path.basename(data))

100%|██████████| 115M/115M [00:11<00:00, 10.3MiB/s]

alexnets.jsonl
nasbench201s.jsonl
mobilenetv3s.jsonl
resnets.jsonl
mobilenetv1s.jsonl
mobilenetv2s.jsonl
proxylessnass.jsonl
vggs.jsonl
shufflenetv2s.jsonl
squeezenets.jsonl
mnasnets.jsonl
densenets.jsonl
googlenets.jsonl


In [10]:
# dataset schema: for each model, we store the: model id, graph in nn-meter IR graph format, latency numbers on four devices
import jsonlines
test_data = datasets[0]
with jsonlines.open(test_data) as data_reader:
    True_lat = []
    Pred_lat = []
    for i, item in enumerate(data_reader):
        print('dict keys:',list(item.keys()))
        print('model id',item['id'])
        print('cpu latency',item['cortexA76cpu_tflite21'])
        print('adreno640gpu latency',item['adreno640gpu_tflite21'])
        print('adreno630gpu latency',item['adreno630gpu_tflite21'])
        print('intelvpu latency',item['myriadvpu_openvino2019r2'])
        print('model graph is stored in nn-meter IR:',item['graph'])
        break

dict keys: ['id', 'cortexA76cpu_tflite21', 'adreno640gpu_tflite21', 'adreno630gpu_tflite21', 'myriadvpu_openvino2019r2', 'graph']
model id alexnet_1356
cpu latency 148.164
adreno640gpu latency 24.4851
adreno630gpu latency 31.932404999999996
intelvpu latency 15.486
model graph is stored in nn-meter IR: {'input_im_0': {'inbounds': [], 'attr': {'name': 'input_im_0', 'type': 'Placeholder', 'output_shape': [[1, 224, 224, 3]], 'attr': {'shape': [1, 224, 224, 3]}, 'input_shape': []}, 'outbounds': ['conv1.conv/Conv2D']}, 'conv1.conv/Conv2D': {'inbounds': ['input_im_0'], 'attr': {'name': 'conv1.conv/Conv2D', 'type': 'Conv2D', 'output_shape': [[1, 56, 56, 63]], 'attr': {'dilations': [1, 1], 'strides': [4, 4], 'data_format': 'NHWC', 'padding': 'VALID', 'kernel_shape': [7, 7], 'weight_shape': [7, 7, 3, 63], 'pads': [0, 0, 0, 0]}, 'input_shape': [[1, 224, 224, 3]]}, 'outbounds': ['conv1.relu.relu/Relu']}, 'conv1.relu.relu/Relu': {'inbounds': ['conv1.conv/Conv2D'], 'attr': {'name': 'conv1.relu.relu/

In [ ]:
# list all supporting latency predictors
predictors = nn_meter.list_latency_predictors()
for p in predictors:
    print(f"[Predictor] {p['name']}: version={p['version']}")

[Predictor] cortexA76cpu_tflite21: version=1.0
[Predictor] adreno640gpu_tflite21: version=1.0
[Predictor] adreno630gpu_tflite21: version=1.0
[Predictor] myriadvpu_openvino2019r2: version=1.0


In [ ]:
# specify basic predictor
predictor_name = 'adreno640gpu_tflite21' # user can change text here to test other predictors
predictor_version = 1.0

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import jsonlines
import nn_meter

# load predictor
predictor = nn_meter.load_latency_predictor(predictor_name, predictor_version)

# view latency prediction demo in one model group of the dataset 
test_data = datasets[0]
with jsonlines.open(test_data) as data_reader:
    True_lat = []
    Pred_lat = []
    for i, item in enumerate(data_reader):
        if i >= 20: # only show the first 20 results to save space
            break
        graph = item["graph"]
        pred_lat = predictor.predict(graph, model_type="nnmeter-ir")
        real_lat = item[predictor_name]
        print(f'[RESULT] {os.path.basename(test_data)}[{i}]: predict: {pred_lat}, real: {real_lat}')

        if real_lat != None:
            True_lat.append(real_lat)
            Pred_lat.append(pred_lat)

if len(True_lat) > 0:
    rmse, rmspe, error, acc5, acc10, _ = nn_meter.latency_metrics(Pred_lat, True_lat)
    print(
        f'[SUMMARY] The first 20 cases from {os.path.basename(test_data)} on {predictor_name}: rmse: {rmse}, 5%accuracy: {acc5}, 10%accuracy: {acc10}'
    )
        

[Result] alexnets.jsonl[0]: predict: 23.447085575244774, real: 24.4851
[Result] alexnets.jsonl[1]: predict: 23.88567577635713, real: 23.9185
[Result] alexnets.jsonl[2]: predict: 29.586297830632216, real: 30.3052
[Result] alexnets.jsonl[3]: predict: 51.12333226388625, real: 52.089
[Result] alexnets.jsonl[4]: predict: 4.937166470494069, real: 5.26442
[Result] alexnets.jsonl[5]: predict: 14.996201148770353, real: 15.2265
[Result] alexnets.jsonl[6]: predict: 9.262593840400985, real: 9.12046
[Result] alexnets.jsonl[7]: predict: 13.912859618198581, real: 14.2242
[Result] alexnets.jsonl[8]: predict: 15.02293612116675, real: 15.2457
[Result] alexnets.jsonl[9]: predict: 12.443609556620194, real: 12.5989
[Result] alexnets.jsonl[10]: predict: 15.971239887611212, real: 15.185
[Result] alexnets.jsonl[11]: predict: 19.46934719077786, real: 20.1434
[Result] alexnets.jsonl[12]: predict: 12.58047633556376, real: 14.4818
[Result] alexnets.jsonl[13]: predict: 18.514081238237033, real: 19.0136
[Result] al

In [ ]:
# apply nn-Meter prediction for all data
for filename in datasets:
    print(f'Start testing {os.path.basename(filename)} ...')
    True_lat = []
    Pred_lat = []
    with jsonlines.open(filename) as data_reader:
        for item in data_reader:
            graph = item["graph"]
            pred_lat = predictor.predict(graph, model_type="nnmeter-ir")
            real_lat = item[predictor_name]
            if real_lat != None:
                True_lat.append(real_lat)
                Pred_lat.append(pred_lat)
    if len(True_lat) > 0:
        rmse, rmspe, error, acc5, acc10, _ = nn_meter.latency_metrics(Pred_lat, True_lat)
        print(
            f'{filename} on {predictor_name}: rmse: {rmse}, 5%accuracy: {acc5}, 10%accuracy: {acc10}'
        )